<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/chisquare(categorical_vars_vs_cluster_ID)051121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade kmodes

In [ ]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [ ]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/dfstd050621.csv")

In [ ]:
data1['AISLE']=data1['AISLE'].astype(str)

In [ ]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [ ]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029976,-0.053250
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092472,-0.022999
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266130,0.004373
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.025112,-0.060323
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077982,-0.062534


In [ ]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [ ]:
data1.shape[0]

16239

In [ ]:
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]

In [ ]:
catColumnsPos

[0, 1, 2, 3, 4, 5, 6]

In [ ]:
# Fit the cluster 
k = 2
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(data1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 66.70981287956238 seconds ---


In [ ]:
data1['Cluster'] = kprototype.labels_

In [ ]:
data1['Cluster']=data1['Cluster'].astype(str)

In [ ]:
data1.describe(include='object')

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,Cluster
count,16239,16239,16239,16239,16239,16239,16239,16239
unique,43,65,133,3,3,14,16,2
top,2,IG45,HTS119,1/1,A,BX,CAT17,1
freq,1117,1572,3763,13428,7073,6351,3986,15722


In [ ]:
data1.describe()

,COSTX,QTYVAR
count,1.623900e+04,1.623900e+04
mean,-1.313664e-16,1.458172e-16
std,1.000031e+00,1.000031e+00
min,-8.424643e+00,-1.598565e-01
25%,-1.223849e-01,-1.051135e-01
50%,2.429776e-02,-5.166240e-02
75%,1.610710e-01,1.016792e-03
max,8.422908e+00,8.091453e+01


In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16239 entries, 0 to 16238
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   AISLE     16239 non-null  object 
 1   IG        16239 non-null  object 
 2   HTS       16239 non-null  object 
 3   PLANNING  16239 non-null  object 
 4   ABC       16239 non-null  object 
 5   UOM       16239 non-null  object 
 6   CAT       16239 non-null  object 
 7   COSTX     16239 non-null  float64
 8   QTYVAR    16239 non-null  float64
 9   Cluster   16239 non-null  object 
dtypes: float64(2), object(8)
memory usage: 1.2+ MB


In [ ]:
from scipy.stats import chi2_contingency

**CAT (categoría)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['CAT'])

In [ ]:
contingency

CAT,CAT1,CAT10,CAT11,CAT12,CAT14,CAT15,CAT16,CAT17,CAT2,CAT3,CAT4,CAT5,CAT6,CAT7,CAT8,CAT9
Cluster,,,,,,,,,,,,,,,,
0,5,6,0,112,196,1,41,33,2,17,7,12,0,28,57,0
1,140,245,1,2445,1942,82,2414,3953,188,502,97,188,7,1194,2164,160


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

9.701235053093032e-71


**AISLE (pasillo de la ubicación de bodega)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['AISLE'])

In [ ]:
contingency

AISLE,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,37,38,39,4,40,41,5,6,7,8,88,9,99
Cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3,7,15,38,3,6,5,9,7,9,3,3,24,3,8,11,5,9,12,8,9,10,9,17,7,12,34,8,6,2,75,3,5,30,4,2,7,19,17,20,0,19,14
1,67,214,621,748,1053,820,780,842,547,577,336,336,1093,243,242,215,85,78,122,74,89,159,138,788,149,157,216,169,99,79,596,183,171,721,117,156,212,174,648,684,83,680,161


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

1.0951566319293859e-73


**IG (item group = grupo de produtos)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['IG'])

In [ ]:
contingency

IG,IG10,IG11,IG12,IG13,IG14,IG15,IG16,IG17,IG18,IG19,IG2,IG20,IG21,IG22,IG25,IG26,IG27,IG28,IG29,IG3,IG30,IG31,IG32,IG33,IG34,IG35,IG36,IG37,IG38,IG39,IG4,IG40,IG41,IG42,IG43,IG44,IG45,IG46,IG47,IG48,IG49,IG5,IG50,IG51,IG52,IG53,IG54,IG55,IG56,IG57,IG58,IG6,IG60,IG61,IG62,IG63,IG64,IG65,IG67,IG69,IG7,IG70,IG71,IG8,IG9
Cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,12,2,7,19,0,22,18,1,1,4,0,0,19,0,7,1,0,5,1,25,26,0,6,5,12,0,0,3,1,0,0,1,2,1,2,11,12,3,2,0,0,0,8,12,6,7,6,3,2,2,3,0,77,1,3,95,0,10,4,28,6,0,6,0
1,97,188,75,333,1049,29,400,111,25,12,106,10,7,377,118,369,255,1,132,31,711,462,6,159,135,546,45,1,606,198,3,27,53,40,115,69,1561,1492,386,85,36,89,17,661,280,705,480,220,390,27,27,48,5,896,3,15,718,18,188,52,260,20,32,103,7


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

5.341254083650463e-87


**HTS (codigo de importación)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['IG'])

In [ ]:
contingency

IG,IG10,IG11,IG12,IG13,IG14,IG15,IG16,IG17,IG18,IG19,IG2,IG20,IG21,IG22,IG25,IG26,IG27,IG28,IG29,IG3,IG30,IG31,IG32,IG33,IG34,IG35,IG36,IG37,IG38,IG39,IG4,IG40,IG41,IG42,IG43,IG44,IG45,IG46,IG47,IG48,IG49,IG5,IG50,IG51,IG52,IG53,IG54,IG55,IG56,IG57,IG58,IG6,IG60,IG61,IG62,IG63,IG64,IG65,IG67,IG69,IG7,IG70,IG71,IG8,IG9
Cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,12,2,7,19,0,22,18,1,1,4,0,0,19,0,7,1,0,5,1,25,26,0,6,5,12,0,0,3,1,0,0,1,2,1,2,11,12,3,2,0,0,0,8,12,6,7,6,3,2,2,3,0,77,1,3,95,0,10,4,28,6,0,6,0
1,97,188,75,333,1049,29,400,111,25,12,106,10,7,377,118,369,255,1,132,31,711,462,6,159,135,546,45,1,606,198,3,27,53,40,115,69,1561,1492,386,85,36,89,17,661,280,705,480,220,390,27,27,48,5,896,3,15,718,18,188,52,260,20,32,103,7


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

5.341254083650463e-87


**PLANNING (codigo de planeación)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['PLANNING'])

In [ ]:
contingency

PLANNING,1/0,1/1,2/0
Cluster,,,
0,48,444,25
1,1541,12984,1197


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is NOT lower than 0.05, so we believe the variables are independent

0.051634058548413075


**ABC (clasificación de inventarios ABC)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['ABC'])

In [ ]:
contingency

ABC,A,B,C
Cluster,,,
0,392,59,66
1,6681,3397,5644


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

2.7114971017737e-50


**UOM (Unit of Measure = unidad)**

In [ ]:
contingency= pd.crosstab(data1['Cluster'], data1['UOM'])

In [ ]:
contingency

UOM,BDL,BG,BX,EA,FT,PC,PK,PR,RL,SET,SKID,SQFT,TB,YD
Cluster,,,,,,,,,,,,,,
0,7,9,350,2,9,39,0,4,22,0,1,3,1,70
1,31,807,6001,58,276,3888,14,1351,752,1,1,606,43,1893


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

3.8800775460683335e-52


In [ ]:
# Prueba sin  CAT11 15 2 6 9
temp = data1[(data1['CAT']!='CAT11') & (data1['CAT']!='CAT15') & (data1['CAT']!='CAT2') & (data1['CAT']!='CAT6') & (data1['CAT']!='CAT9')]

In [ ]:
contingency= pd.crosstab(temp['Cluster'], temp['CAT'])

In [ ]:
contingency

CAT,CAT1,CAT10,CAT12,CAT14,CAT16,CAT17,CAT3,CAT4,CAT5,CAT7,CAT8
Cluster,,,,,,,,,,,
0,5,6,112,196,41,33,17,7,12,28,57
1,140,245,2445,1942,2414,3953,502,97,188,1194,2164


In [ ]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contingency) 
# Print the p-value
print(p)
# Now, p < 0.05 is the usual test for dependence. In this case p is lower than 0.05, so we believe the variables are dependent

7.060486378066381e-71


def chi2_contingency(observed, correction=True, lambda_=None)
Chi-square test of independence of variables in a contingency table.

This function computes the chi-square statistic and p-value for the
hypothesis test of independence of the observed frequencies in the
contingency table [1]_ observed.  The expected frequencies are computed
based on the marginal sums under the assumption of independence; see
scipy.stats.contingency.expected_freq.  The number of degrees of
freedom is (expressed using numpy functions and attributes):

dof = observed.size - sum(observed.shape) + observed.ndim - 1
Parameters

observed : array_like
    The contingency table. The table contains the observed frequencies
(i.e. number of occurrences) in each category.  In the two-dimensional
case, the table is often described as an "R x C table".
correction : bool, optional
    If True, *and* the degrees of freedom is 1, apply Yates' correction
for continuity.  The effect of the correction is to adjust each
observed value by 0.5 towards the corresponding expected value.
lambda_ : float or str, optional.
    By default, the statistic computed in this test is Pearson's
chi-squared statistic [2]_.  lambda_ allows a statistic from the
Cressie-Read power divergence family [3]_ to be used instead.  See
power_divergence for details.

Returns

chi2 : float
    The test statistic.
p : float
    The p-value of the test
dof : int
    Degrees of freedom
expected : ndarray, same shape as observed
    The expected frequencies, based on the marginal sums of the table.

See Also

contingency.expected_freq
fisher_exact
chisquare
power_divergence

Notes

An often quoted guideline for the validity of this calculation is that
the test should be used only if the observed and expected frequencies
in each cell are at least 5.

This is a test for the independence of different categories of a
population. The test is only meaningful when the dimension of
observed is two or more.  Applying the test to a one-dimensional
table will always result in expected equal to observed and a
chi-square statistic equal to 0.

This function does not handle masked arrays, because the calculation
does not make sense with missing values.

Like stats.chisquare, this function computes a chi-square statistic;
the convenience this function provides is to figure out the expected
frequencies and degrees of freedom from the given contingency table.
If these were already known, and if the Yates' correction was not
required, one could use stats.chisquare.  That is, if one calls:

chi2, p, dof, ex = chi2_contingency(obs, correction=False)
then the following is true:

(chi2, p) == stats.chisquare(obs.ravel(), f_exp=ex.ravel(),
                             ddof=obs.size - 1 - dof)
The lambda_ argument was added in version 0.13.0 of scipy.

References

Examples

A two-way example (2 x 3):

>>> from scipy.stats import chi2_contingency
>>> obs = np.array([[10, 10, 20], [20, 20, 20]])
>>> chi2_contingency(obs)
(2.7777777777777777,
 0.24935220877729619,
 2,
 array([[ 12.,  12.,  16.],
        [ 18.,  18.,  24.]]))
Perform the test using the log-likelihood ratio (i.e. the "G-test")
instead of Pearson's chi-squared statistic.

>>> g, p, dof, expctd = chi2_contingency(obs, lambda_="log-likelihood")
>>> g, p
(2.7688587616781319, 0.25046668010954165)
A four-way example (2 x 2 x 2 x 2):

>>> obs = np.array(
...     [[[[12, 17],
...        [11, 16]],
...       [[11, 12],
...        [15, 16]]],
...      [[[23, 15],
...        [30, 22]],
...       [[14, 17],
...        [15, 16]]]])
>>> chi2_contingency(obs)
(8.7584514426741897,
 0.64417725029295503,
 11,
 array([[[[ 14.15462386,  14.15462386],
          [ 16.49423111,  16.49423111]],
         [[ 11.2461395 ,  11.2461395 ],
          [ 13.10500554,  13.10500554]]],
        [[[ 19.5591166 ,  19.5591166 ],
          [ 22.79202844,  22.79202844]],
         [[ 15.54012004,  15.54012004],
          [ 18.10873492,  18.10873492]]]]))